# <center> <img src="figs/logo_facens.png" alt="Logo Facens" width="110" align="left"/>  <br/> <center>Facens<br/>
</p>

<br/>
  
<font size="3"><center>Victor Hugo Soares Tavares, 171689</center></font>




# Aprendizado de Máquinas
### 2022PGS2M1
----

## <center> Atividade Ensemble </center>

    
Leitura do dataset do data_cancer

Normalização dos dados

Split do Dataset

TESTES:
- Resultado (classification report) com Adaboost
- Resultado (classification report) com Bagging
- Resultado (classification report) do GridSearch do Voting com os modelos
KNN
DecisionTrees
SVM

## Importando Libs

In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, precision_score, recall_score, \
                            f1_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import datasets
import random
from sklearn.pipeline import Pipeline
from glob import glob
from tqdm.notebook import tqdm

# Nossas ferramentas
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, cross_val_score

# Nossos Algoritmos
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.svm import SVC

# Ensemble
from sklearn.ensemble import VotingClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier

<strong>Leitura do dataset<strong/>

In [6]:
cancer = pd.read_csv('datasets/data_cancer.csv')

In [7]:
cancer.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


<strong>Normalização dos dados / Split do Dataset<strong/>

In [9]:
X = cancer.iloc[:, :-1].values
y = cancer.iloc[:, -1].values

In [12]:
# Separar em treino e validação, normalizar os dados
sc = StandardScaler()
X_norm = sc.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_norm, y, random_state=42)

<strong>Resultado (classification report) com Adaboost<strong/>

In [14]:
ada_model = AdaBoostClassifier()
ada_model.fit(X_train, y_train)
y_pred_ada = ada_model.predict(X_test)

In [15]:
print(classification_report(y_test, y_pred_ada))

              precision    recall  f1-score   support

           2       0.93      0.99      0.96       103
           4       0.98      0.88      0.93        68

    accuracy                           0.95       171
   macro avg       0.96      0.94      0.94       171
weighted avg       0.95      0.95      0.95       171



<strong>Resultado (classification report) com Bagging<strong/>

In [17]:
bag_model = BaggingClassifier()
bag_model.fit(X_train, y_train)
y_pred_bag = bag_model.predict(X_test)

In [18]:
print(classification_report(y_test, y_pred_bag))

              precision    recall  f1-score   support

           2       0.95      0.98      0.97       103
           4       0.97      0.93      0.95        68

    accuracy                           0.96       171
   macro avg       0.96      0.95      0.96       171
weighted avg       0.96      0.96      0.96       171



<strong>Resultado (classification report) do GridSearch do Voting com os modelos
KNN
DecisionTrees
SVM<strong/>

In [19]:
voting_classifier = VotingClassifier([('KNN', KNeighborsClassifier()), 
                                      ('TREE', DecisionTreeClassifier()), 
                                      ('SVM', SVC())])

In [27]:
params_grid = {
    'KNN__n_neighbors': [5, 7, 9],
    'TREE__max_depth': [1, 60, 100],
    'TREE__criterion': ['gini', 'entropy'],
    'SVM__C': [1,10,100],
    'SVM__gamma': [1,10,100],
    'SVM__kernel': ['linear', 'rbf']
}

In [28]:
grid_classifier = GridSearchCV(voting_classifier, param_grid = params_grid, verbose=1)

In [29]:
grid_classifier.fit(X_train, y_train)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


GridSearchCV(estimator=VotingClassifier(estimators=[('KNN',
                                                     KNeighborsClassifier()),
                                                    ('TREE',
                                                     DecisionTreeClassifier()),
                                                    ('SVM', SVC())]),
             param_grid={'KNN__n_neighbors': [5, 7, 9], 'SVM__C': [1, 10, 100],
                         'SVM__gamma': [1, 10, 100],
                         'SVM__kernel': ['linear', 'rbf'],
                         'TREE__criterion': ['gini', 'entropy'],
                         'TREE__max_depth': [1, 60, 100]},
             verbose=1)

In [30]:
grid_classifier.best_params_

{'KNN__n_neighbors': 9,
 'SVM__C': 100,
 'SVM__gamma': 1,
 'SVM__kernel': 'linear',
 'TREE__criterion': 'gini',
 'TREE__max_depth': 60}

In [31]:
print(round(grid_classifier.best_score_, 5))

0.9668


In [32]:
y_pred = grid_classifier.predict(X_test)

In [33]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           2       0.94      0.99      0.97       103
           4       0.98      0.91      0.95        68

    accuracy                           0.96       171
   macro avg       0.96      0.95      0.96       171
weighted avg       0.96      0.96      0.96       171

